In [38]:
import spacy
import re
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy import displacy
nlp = spacy.load('pl_core_news_sm')

In [5]:
#Zczytywanie słów ze stringa

intro_text = ('We wrześniu roku 1841 w okolicach miasteczka Hagenów na terenie Meklemburgii ' 
              'zostały odkryte dwa groby szkieletowe datowane na okres wczesnorzymski '
              '(II w.po Chr.)')
intro_doc = nlp(intro_text)
print([token.text for token in intro_doc])


['We', 'wrześniu', 'roku', '1841', 'w', 'okolicach', 'miasteczka', 'Hagenów', 'na', 'terenie', 'Meklemburgii', 'zostały', 'odkryte', 'dwa', 'groby', 'szkieletowe', 'datowane', 'na', 'okres', 'wczesnorzymski', '(', 'II', 'w.po', 'Chr', '.', ')']


In [8]:
#Zczytywanie słów z dokumentu teksowego

file_name = 'intro.txt'
intro_file_text = open(file_name).read()
intro_file_doc = nlp(intro_file_text)
print ([token.text for token in intro_file_doc])

['We', 'wrzeÅ›niu', 'roku', '1841', 'w', 'okolicach', 'miasteczka', 'HagenÃ³w', 'na', 'terenie', 'Meklemburgii', '\n', 'zostaÅ', '‚', 'y', 'odkryte', 'dwa', 'groby', 'szkieletowe', 'datowane', 'na', 'okres', 'wczesnorzymski', '(', 'II', 'w', '.', '\n', 'po', 'Chr', '.', ')']


In [10]:
#Wykrywanie zdań

text = ('Wojny galijskie to czas ważnych zmian w rzymskiej wojskowości, w dużej części będących zasługą Juliusza Cezara. Nie tylko zwiększył on żołd, ale także zadbał o ściślejsze podporządkowanie żołnierzy dowódcy. Zmiany w tym okresie objęły sposoby zdobywania miast, a nawet kwestie obyczajowe – w nieprzystępnym klimacie Cezar pozwolił żołnierzom na... noszenie spodni.')
doc = nlp(text)
sentences = list(doc.sents)
print (len(sentences))
for sentence in sentences:
    print (sentence)

4
Wojny galijskie to czas ważnych zmian w rzymskiej wojskowości, w dużej części będących zasługą Juliusza Cezara.
Nie tylko zwiększył on żołd, ale także zadbał o ściślejsze podporządkowanie żołnierzy dowódcy.
Zmiany w tym okresie objęły sposoby zdobywania miast, a nawet kwestie obyczajowe – w nieprzystępnym klimacie Cezar pozwolił żołnierzom na...
noszenie spodni.


In [17]:
#Uwzględnienie trzykropków 

#nie działa ;_;

def set_custom_boundaries(doc):
     for token in doc[:-1]:
         if token.text == '...':
             doc[token.i+1].is_sent_start = True
     return doc

c_nlp = spacy.load('pl_core_news_sm')
c_nlp.add_pipe(set_custom_boundaries, before='parser')
c_doc = c_nlp(text)
c_sentences = list(c_doc.sents)
print (len(c_sentences))
for sentence in c_sentences:
    print (sentence)


4
Wojny galijskie to czas ważnych zmian w rzymskiej wojskowości, w dużej części będących zasługą Juliusza Cezara.
Nie tylko zwiększył on żołd, ale także zadbał o ściślejsze podporządkowanie żołnierzy dowódcy.
Zmiany w tym okresie objęły sposoby zdobywania miast, a nawet kwestie obyczajowe – w nieprzystępnym klimacie Cezar pozwolił żołnierzom na...
noszenie spodni.


In [14]:
#Identyfikacja tokenów

for token in intro_doc:
    print (token, token.idx)

We 0
wrześniu 3
roku 12
1841 17
w 22
okolicach 24
miasteczka 34
Hagenów 45
na 53
terenie 56
Meklemburgii 64
zostały 77
odkryte 85
dwa 93
groby 97
szkieletowe 103
datowane 115
na 124
okres 127
wczesnorzymski 133
( 148
II 149
w.po 152
Chr 157
. 160
) 161


In [15]:
#Atrybuty tokenów

for token in intro_doc:
     print (token, token.idx, token.text_with_ws,
            token.is_alpha, token.is_punct, token.is_space,
            token.shape_, token.is_stop)


We 0 We  True False False Xx True
wrześniu 3 wrześniu  True False False xxxx False
roku 12 roku  True False False xxxx True
1841 17 1841  False False False dddd False
w 22 w  True False False x True
okolicach 24 okolicach  True False False xxxx False
miasteczka 34 miasteczka  True False False xxxx False
Hagenów 45 Hagenów  True False False Xxxxx False
na 53 na  True False False xx True
terenie 56 terenie  True False False xxxx False
Meklemburgii 64 Meklemburgii  True False False Xxxxx False
zostały 77 zostały  True False False xxxx False
odkryte 85 odkryte  True False False xxxx False
dwa 93 dwa  True False False xxx True
groby 97 groby  True False False xxxx False
szkieletowe 103 szkieletowe  True False False xxxx False
datowane 115 datowane  True False False xxxx False
na 124 na  True False False xx True
okres 127 okres  True False False xxxx False
wczesnorzymski 133 wczesnorzymski  True False False xxxx False
( 148 ( False True False ( False
II 149 II  True False False XX False
w.po

In [19]:
#Własny token

prefix_re = spacy.util.compile_prefix_regex(c_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(c_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                     suffix_search=suffix_re.search,
                     infix_finditer=infix_re.finditer,
                     token_match=None
                     )

c_nlp.tokenizer = customize_tokenizer(c_nlp)
c_tokenizer_doc = c_nlp(intro_text)
print([token.text for token in c_tokenizer_doc])


['We', 'wrześniu', 'roku', '1841', 'w', 'okolicach', 'miasteczka', 'Hagenów', 'na', 'terenie', 'Meklemburgii', 'zostały', 'odkryte', 'dwa', 'groby', 'szkieletowe', 'datowane', 'na', 'okres', 'wczesnorzymski', '(', 'II', 'w.po', 'Chr', '.', ')']


In [22]:
#Najczęstsze słowa w języku polskim (stop-słowa)

stopwords = spacy.lang.pl.stop_words.STOP_WORDS
print(len(stopwords))

for stop_word in list(stopwords)[:15]:
    print(stop_word)

381
cała
mimo
o
twój
viii
raz
cokolwiek
dziś
wszyscy
totobą
gdzieś
zawsze
przede
dzis
kiedy


In [23]:
#Usunięcie stop-słów z tekstu

for token in doc:
    if not token.is_stop:
        print (token)

Wojny
galijskie
czas
ważnych
zmian
rzymskiej
wojskowości
,
dużej
części
będących
zasługą
Juliusza
Cezara
.
zwiększył
żołd
,
zadbał
ściślejsze
podporządkowanie
żołnierzy
dowódcy
.
Zmiany
okresie
objęły
sposoby
zdobywania
miast
,
kwestie
obyczajowe
–
nieprzystępnym
klimacie
Cezar
pozwolił
żołnierzom
.
.
.
noszenie
spodni
.


In [24]:
#Upraszczanie formy (lematyzacja)
doc = nlp(text)
for token in doc:
    print(token,' - ', token.lemma_)

Wojny  -  wojna
galijskie  -  galijski
to  -  to
czas  -  czas
ważnych  -  ważny
zmian  -  zmiana
w  -  w
rzymskiej  -  rzymski
wojskowości  -  wojskowość
,  -  ,
w  -  w
dużej  -  duży
części  -  część
będących  -  być
zasługą  -  zasługa
Juliusza  -  Juliusz
Cezara  -  cezar
.  -  .
Nie  -  nie
tylko  -  tylko
zwiększył  -  zwiększyć
on  -  on
żołd  -  żołd
,  -  ,
ale  -  ale
także  -  także
zadbał  -  zadbać
o  -  o
ściślejsze  -  ścisły
podporządkowanie  -  podporządkowanie
żołnierzy  -  żołnierz
dowódcy  -  dowódca
.  -  .
Zmiany  -  zmiana
w  -  w
tym  -  ten
okresie  -  okres
objęły  -  objąć
sposoby  -  sposób
zdobywania  -  zdobywania
miast  -  miasto
,  -  ,
a  -  a
nawet  -  nawet
kwestie  -  kwestia
obyczajowe  -  obyczajowy
–  -  –
w  -  w
nieprzystępnym  -  przystępny
klimacie  -  klimat
Cezar  -  cezar
pozwolił  -  pozwolić
żołnierzom  -  żołnierz
na  -  na
.  -  .
.  -  .
.  -  .
noszenie  -  noszenie
spodni  -  spodnie
.  -  .


In [34]:
#Analiza tekstu pod względem występowania słów

an_text = ('W omawianym okresie siły morskie Cezara nie były zbyt liczne. Rywalizacja z plemionem Wenetów wiodącym prym na północno-zachodnim wybrzeżu Galii zmusiła go jednak do budowy stosunkowo silnej floty. Wspomina o tym Kasjusz Dion: W głębi kraju budował okręty, które, jak słyszał są pożyteczne podczas przypływów i odpływów (...) kazał sprowadzić z prowincji wioślarzy, pościągać żeglarzy i sterników28. Nowo wybudowana flota rzymska wzięła niebawem udział w wielkiej bitwie morskiej u ujścia Loary, rozegranej w 56 r. p.n.e. Podczas niej prokonsul Galii zastosował wynalazek techniczny, zwany harpago bądź harpax, kładąc w ten sposób duże zasługi na polu unowocześnienia wyposażenia okrętów rzymskich. Była to przymocowana do liny i zakończona hakiem żerdź wystrzeliwana z katapulty. Mechanizm działania wspomnianego wynalazku, umożliwiającego zholowanie wrogiego okrętu w rejon własnego, tak opisuje Cezar: Jedno tylko urządzenie zastosowane przez naszych było bardzo użyteczne, mianowicie dobrze na końcu wyostrzone sierpy, nasadzone i przymocowane do żerdzi, przypominające wyglądem osęki murowe. Gdy liny, którymi reje były przymocowane do masztów, zostały nimi zahaczone i przyciągnięte, po puszczeniu w ruch naszego statku przy pomocy wioseł ulegały rozerwaniu29. Niewykluczone, że po ujarzmieniu Wenetów flotę rzymską zasiliły okręty zbuntowanego plemienia, które według Cezara miały: (...) dna bardziej płaskie (...) aby mogły łatwiej wyzwalać się z mielizn i odpływów; dzioby równie wysoko podniesione jak rufy przystosowane do wysokiej fali i mórz; okręty były w całości zbudowane z dębiny. W dziedzinie taktyki walk morskich nie wprowadzono większych zmian. Flota morska Cezara, zbudowana głównie w galijskich stoczniach, stosowała podczas bitew nadal taktykę wypracowaną w okresie wczesnej republiki. Poza wykorzystaniem floty wojennej, w powszechnym użyciu pozostawały wciąż statki transportowe, ściągane z Morza Śródziemnego. Podczas swej wyprawy do Brytanii w 55 r. p.n.e. Cezar dysponował ok. 800 jednostkami tego typu. To dzięki niej przeprawa legionów na nowo odkrytą przez Rzymian wyspę zakończyła się powodzeniem.')

an_doc = nlp(an_text)
words = [token.text for token in an_doc
        if not token.is_punct]
relevant_words = [token.text for token in an_doc
        if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
rel_freq = Counter(relevant_words)
print ('Top 5: ', word_freq.most_common(5))
print ('Top 5 bez stop-słów', rel_freq.most_common(5))

Top 5:  [('w', 10), ('i', 7), ('z', 6), ('do', 6), ('na', 4)]
Top 5 bez stop-słów [('Cezara', 3), ('okręty', 3), ('okresie', 2), ('Wenetów', 2), ('Galii', 2)]


In [35]:
#Rozpozawanie słów jako części mowy

for token in doc:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))


Wojny SUBST NOUN None
galijskie ADJ ADJ adjective
to PRED VERB None
czas SUBST NOUN None
ważnych ADJ ADJ adjective
zmian SUBST NOUN None
w PREP ADP None
rzymskiej ADJ ADJ adjective
wojskowości SUBST NOUN None
, INTERP PUNCT None
w PREP ADP None
dużej ADJ ADJ adjective
części SUBST NOUN None
będących PACT VERB None
zasługą SUBST NOUN None
Juliusza SUBST NOUN None
Cezara SUBST NOUN None
. INTERP PUNCT None
Nie CONJ CCONJ conjunction
tylko CONJ CCONJ conjunction
zwiększył PRAET VERB None
on PPRON3 PRON None
żołd SUBST NOUN None
, INTERP PUNCT None
ale CONJ CCONJ conjunction
także CONJ CCONJ conjunction
zadbał PRAET VERB None
o PREP ADP None
ściślejsze ADJ ADJ adjective
podporządkowanie GER NOUN None
żołnierzy SUBST NOUN None
dowódcy SUBST NOUN None
. INTERP PUNCT None
Zmiany SUBST NOUN None
w PREP ADP None
tym ADJ ADJ adjective
okresie SUBST NOUN None
objęły PRAET VERB None
sposoby SUBST NOUN None
zdobywania GER NOUN None
miast SUBST NOUN None
, INTERP PUNCT None
a CONJ CCONJ conjunction


In [37]:
#Tylko rzeczowniki

nouns = []
for token in doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
print (nouns)

[Wojny, czas, zmian, wojskowości, części, zasługą, Juliusza, Cezara, żołd, podporządkowanie, żołnierzy, dowódcy, Zmiany, okresie, sposoby, zdobywania, miast, kwestie, klimacie, Cezar, żołnierzom, noszenie, spodni]


In [ ]:
#Wizualizacja

displacy.render(doc, style='dep', jupyter=True)